# Homework 04

**Name:** -- Maria Alexa Ruiz Madero --

**e-mail:** -- maria.ruiz@alumnos.udg.mx --

# MODULES

In [171]:
# Load modules
import panel as pn 
pn.extension('plotly')

import panel.widgets as pnw

import pandas as pd

import numpy as np

import plotly.graph_objects as go

import math

from scipy.stats import wrapcauchy 
from scipy.stats import levy_stable

In [50]:
pn.__version__

'1.6.1'

# CLASSES

In [172]:
class Vec2d(object):
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y           
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)
    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [180]:
#Define a function called BM
def bm_2d(n_steps=1000, speed=1, s_x_pos=0, s_y_pos=0):
    BM_2d_df = pd.DataFrame({'x_pos': [s_x_pos], 'y_pos': [s_y_pos]})
    rw_ = [(s_x_pos, s_y_pos)]
    
    for _ in range(n_steps):
        angle = np.random.uniform(0, 2 * np.pi)
        velocity = speed * np.array([np.cos(angle), np.sin(angle)])

        updated_x = BM_2d_df.x_pos.iloc[-1] + velocity[0]
        updated_y = BM_2d_df.y_pos.iloc[-1] + velocity[1]
        
        BM_2d_df = pd.concat(
            [BM_2d_df, pd.DataFrame({'x_pos': [updated_x], 'y_pos': [updated_y]})], 
            ignore_index=True
        )

        rw_.append((updated_x, updated_y))  

    return BM_2d_df

#Define a function called CRW 

def correlated_random_walk(n_steps= 100, speed=6, s_x_pos=0, s_y_pos=0, cauchy_coef=0.5):
    trajectory = pd.DataFrame({'x_pos':[s_x_pos], 'y_pos':[s_y_pos]})
    angle = np.random.uniform(-np.pi, np.pi)
    
#The walk
    for i in range (n_steps-1): 
        angle += wrapcauchy.rvs(cauchy_coef)
        updated_x = trajectory.iloc[-1].x_pos + speed * np.cos(angle)
        updated_y = trajectory.iloc[-1].y_pos + speed * np.sin(angle)
        
        trajectory = pd.concat([trajectory, pd.DataFrame({'x_pos': [updated_x], 'y_pos':[updated_y]})], ignore_index=True)

    return trajectory

#Define a function called levy flight

def levy_flight(n_steps=100, speed=6, s_x_pos=0, s_y_pos=0, alpha=1.5, cauchy_coef=0.5):
    trajectory = pd.DataFrame({'x_pos':[s_x_pos], 'y_pos': [s_y_pos]})
    angles = np.random.uniform(0, 2*np.pi, n_steps)
    step_lengths = levy_stable.rvs(alpha, cauchy_coef, size=n_steps) * speed
 
    for i in range(n_steps -1): 
        updated_x = trajectory.iloc[-1].x_pos + step_lengths[i] * np.cos(angles[i]) 
        updated_y = trajectory.iloc[-1].y_pos + step_lengths[i] * np.sin(angles[i])
        
        trajectory = pd.concat([trajectory, pd.DataFrame({'x_pos': [updated_x], 'y_pos': [updated_y]})], ignore_index=True)

    return trajectory

#Funcition to calculate the cumulative path length 
def calculate_path_length(df): 
    if isinstance(df, np.ndarray):
        df= pd.DataFrame(df, columns=['x_pos', 'y_pos'])
        
    dx = np.diff(df['x_pos'])
    dy = np.diff(df['y_pos'])
    distances = np.sqrt(dx**2 + dy**2)
    return np.cumsum(distances)

#Calculate Mean Squared Displacement(MSD)
def calculate_msd(df):
    displacements = (df['x_pos']- df['x_pos'].iloc[0])**2 + (df['y_pos']- df['y_pos'].iloc[0])**2
    return np.cumsum(displacements)/np.arange(1, len(displacements)+ 1)

#MSD with euclidean distance 
def compute_msd(df):
    MSD_list =[]
    for tau in range (1, df.shape[0]):
        displacement_vec = np.array([
            np.linalg.norm(df.iloc[i -tau], df.iloc[i])#calculates euclidean 
            for i in range(tau, df.shape[0])
        ])
        MSD_list.append(np.mean(displacement_vec**2))
    return np.asarray(MSD_list)

#function to calculate turning angles
def calculate_turning_angles(df):
    dx = np.diff(df['x_pos'])
    dy = np.diff(df['y_pos'])
    angles = np.arctan2(dy, dx)
    turning_angles = np.diff(angles)
    return turning_angles

def plot_trajec(rw, n_steps=1000):
    times = np.linspace(0,1, len(rw))
    fig_traj_rw = go.Figure()
    
    fig_traj_rw.add_trace(go.Scatter3d(
        x = rw[:,0],
        y = rw[:,1],
        z = times,
        marker = dict(size=2),
        line = dict(color='red', width=2),
        mode = 'lines',
        name = f'steps = {len(rw)}',
        showlegend = True
    ))


    fig_traj_rw.update_layout(
        title='Trajectory',
        scene=dict(
            xaxis_title='X Position',
            yaxis_title='Y Position',
            zaxis_title='Time'
        )
    )
    
    return fig_traj_rw

def plot_metrics_(pl, n_steps=1000, metric_name='Metric'):
    fig_metrics_rw = go.Figure()
    
    fig_metrics_rw.add_trace(go.Scatter(
        x = np.arange(len(pl)),
        y = pl,
        marker = dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = f"{metric_name} (Step={len(pl)})",
        showlegend = True
    ))

    fig_metrics_rw.update_layout(
    title=f"{metric_name}",
    xaxis_title="Step",
    yaxis_title=metric_name
    )
    
    return fig_metrics_rw

# PANEL

In [174]:
#The basic structure of the panel is defined

panel_layout= pn.Column(
    pn.pane.Markdown('Panel of Random Walks'),
    pn.pane.Markdown('Parameters Configuration'),
    pn.pane.Markdown('Viewing Walks')
)

# WIDGETS

In [175]:
# Widgets are defined for selecting parameters
n_steps = pnw.IntSlider(name='Number of Steps', width=400, value=50, step=10, start=10, end=100)
s_x_pos = pnw.IntInput(name='Initial Position in x', value= 10, step=10, start=-100, end=100)
s_y_pos = pnw.IntInput(name='Initial Position in Y', value=10, step=10, start=-100, end=100) 
cauchy_coef = pnw.FloatSlider(name='Cauchy Coefficient(CRW)', width=400, value=0.5, step=0.1, start=0.1, end=1.0)


# DATAFRAME

In [181]:
@pn.depends(n_steps, s_x_pos, s_y_pos, walk_type, cauchy_coef, alpha)
def create_df(n_steps, s_x_pos, s_y_pos, walk_type, cauchy_coef, alpha):
    if walk_type == 'BM':
        return bm_2d(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos)
    elif walk_type == 'CRW':
        return correlated_random_walk(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos, cauchy_coef=cauchy_coef)
    elif walk_type == 'LF':
        return levy_flight(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos, alpha=alpha)


# TRAJECTORIES

In [182]:

# Plot grafics
@pn.depends(n_steps, s_x_pos, s_y_pos)
def plot_traj_bm(n_steps, s_x_pos, s_y_pos):
    df = bm_2d(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos)
    
    return plot_trajec(df.to_numpy(), n_steps)

@pn.depends(n_steps, s_x_pos, s_y_pos, cauchy_coef)
def plot_traj_crw(n_steps, s_x_pos, s_y_pos, cauchy_coef):
    df = correlated_random_walk(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos, cauchy_coef=cauchy_coef)
    
    return plot_trajec(df.to_numpy(), n_steps)

@pn.depends(n_steps, s_x_pos, s_y_pos, cauchy_coef, alpha)
def plot_traj_lf(n_steps, s_x_pos, s_y_pos, cauchy_coef, alpha):
    df = levy_flight(n_steps, s_x_pos=s_x_pos, s_y_pos=s_y_pos, cauchy_coef=cauchy_coef, alpha=alpha)
    
    return plot_trajec(df.to_numpy(), n_steps)

# Plot metrics
@pn.depends(n_steps, s_x_pos, metric)
def plot_metrics_bm(n_steps, s_x_pos, metric):
    df = bm_2d(n_steps, s_x_pos=s_x_pos)
    
    if(metric == 0):
        pl = calculate_path_length(df)
        pl = calculate_path_length(df)
    elif(metric == 1):
       
        pl = calculate_msd(df)
    return plot_metrics_(pl, n_steps)

@pn.depends(n_steps, s_x_pos, cauchy_coef, metric)
def plot_metrics_crw(n_steps, s_x_pos, cauchy_coef, metric):
    df = correlated_random_walk(n_steps, s_x_pos=s_x_pos, cauchy_coef=cauchy_coef)
    
    if(metric == 0):
        pl = calculate_path_length(df)
        pl = calculate_path_length(df)
    elif(metric == 1):
        
        pl = calculate_msd(df)
    return plot_metrics_(pl, n_steps)

@pn.depends(n_steps, s_x_pos, cauchy_coef, alpha, metric)
def plot_metrics_lf(n_steps, s_x_pos, cauchy_coef, alpha, metric):
    df = levy_flight(n_steps, s_x_pos=s_x_pos, cauchy_coef=cauchy_coef, alpha=alpha)
    
    if(metric == 0):
       
        pl = calculate_path_length(df)
    elif(metric == 1):
        
        pl = calculate_msd(df)
    return plot_metrics_(pl, n_steps)

# View grafics
p_bm = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, s_y_pos, metric),
    pn.Row('Grafics'),
    pn.Row(plot_traj_bm,plot_metrics_bm)
)

p_crw = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, s_y_pos, metric, cauchy_coef),
    pn.Row('Grafics'),
    pn.Row(plot_traj_crw,plot_metrics_crw)
)

p_lf = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, metric, s_y_pos, cauchy_coef, alpha),
    #pn.Row(alpha),
    pn.Row('Grafics'),
    pn.Row(plot_traj_lf,plot_metrics_lf)
)

tabs = pn.Tabs()

tabs.extend([
    ('BM', p_bm),
    ('CRW', p_crw),
    ('Levy', p_lf)
])

tabs



Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=100, name='Number of Steps', start=10, step=10, value=50, width=400)
            [1] IntInput(end=100, name='Initial Position in x', start=-100, step=10, value=10)
            [2] IntInput(end=100, name='Initial Position in Y', start=-100, step=10, value=10)
            [3] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=1)
        [2] Row
            [0] Markdown(str)
        [3] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False)
    [1] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=100, name='Number of Steps', start=10, step=10, value=50, width=400)
            [1] IntInput(end=100, name='Initial Position in x', start=-100, step=10, value=10)
            [2] IntInput(end=100, name='Initial Position in Y', start=-100, step=10, value=10)
            [3] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=1)
            [4] FloatSlider(name='Cauchy Coefficient(CRW)', start=0.1, value=0.5, width=400)
        [2] Row
            [0] Markdown(str)
        [3] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=100, name='Number of Steps', start=10, step=10, value=50, width=400)
            [1] IntInput(end=100, name='Initial Position in x', start=-100, step=10, value=10)
            [2] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=1)
            [3] IntInput(end=100, name='Initial Position in Y', start=-100, step=10, value=10)
            [4] FloatSlider(name='Cauchy Coefficient(CRW)', start=0.1, value=0.5, width=400)
            [5] FloatSlider(end=2.0, name='Levy Exponent (LF)', start=0.5, value=1.5, width=400)
        [2] Row
            [0] Markdown(str)
        [3] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False)